In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import time

# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 512 # what is the maximum context length for predictions?
max_iters = 1500
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 192
n_head = 6
n_layer = 8
dropout = 0.2


"# hyperparameters\nbatch_size = 64 # how many independent sequences will we process in parallel?\nblock_size = 256 # what is the maximum context length for predictions?\nmax_iters = 1500\neval_interval = 500\nlearning_rate = 3e-4\ndevice = 'cuda' if torch.cuda.is_available() else 'cpu'\neval_iters = 200\nn_embd = 384\nn_head = 6\nn_layer = 6\ndropout = 0.2"

In [ ]:
!pip uninstall -y nltk



Found existing installation: nltk 3.9.1
Uninstalling nltk-3.9.1:
  Successfully uninstalled nltk-3.9.1


In [ ]:
!pip install nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 32.5 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:

# ------------
# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# Uploading the trainset
from google.colab import files
uploaded = files.upload ()
file_name = list(uploaded.keys())[0]
with open(file_name, 'r', encoding='utf-8') as f:
    melodies = f.read()

Saving inputMelodiesAugmented_new.txt to inputMelodiesAugmented_new.txt


In [ ]:
"""from google.colab import files
uploaded = files.upload ()
file_name = list(uploaded.keys())[0]
with open(file_name, 'r', encoding='utf-8') as f:
    test_childspeech = f.read()"""

"from google.colab import files\nuploaded = files.upload ()\nfile_name = list(uploaded.keys())[0]\nwith open(file_name, 'r', encoding='utf-8') as f:\n    test_childspeech = f.read()"

In [ ]:
"""from google.colab import files
uploaded = files.upload ()
file_name = list(uploaded.keys())[0]
with open(file_name, 'r', encoding='utf-8') as f:
    test_shakesp = f.read()"""

"from google.colab import files\nuploaded = files.upload ()\nfile_name = list(uploaded.keys())[0]\nwith open(file_name, 'r', encoding='utf-8') as f:\n    test_shakesp = f.read()"

In [ ]:
from nltk.tokenize import word_tokenize


In [ ]:
# Properties of the dataset
chars = sorted(list(set(melodies)))
vocab_size = len(chars)
print (chars)

['\n', '2', '3', '4', '5', '6', '7', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'R', 'a', 'c', 'd', 'e', 'f', 'g', 'h', 'q', 's', 'w']


In [ ]:
# Create mappings
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}


In [ ]:
encode = lambda s: [stoi[c] for c in s]  # Encoder: string -> list of integers
decode = lambda l: ''.join([itos[i] for i in l])  # Decoder: list of integers -> string"""

In [ ]:
data = torch.tensor(encode(melodies), dtype=torch.long)
n = int(0.9*len(data))  # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [ ]:
# Dataset properties
"""wort_tokenize = word_tokenize(text.lower())
vocabulary = set(wort_tokenize)
print (len(vocabulary))
length = len (text)
print (length)

# All datasets combined to extract all unique characters
all_text = text + test_childspeech + test_shakesp
#all_text = text + test_childspeech

# List of all unique characters
chars = sorted(list(set(all_text)))
print("Unique characters:", chars)

# Vocabulary size
vocab_size = len(chars)
print("Vocabulary size:", vocab_size)

# Create mappings
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]  # Encoder: string -> list of integers
decode = lambda l: ''.join([itos[i] for i in l])  # Decoder: list of integers -> string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
#test_data_1 = torch.tensor(encode(test_childspeech), dtype=torch.long)
test_data_2 = torch.tensor(encode(test_shakesp), dtype=torch.long)
#n = int(0.9*len(data))
#train_data = data[:n]
#val_data = data[n:]
train_data = data
#val_data = test_data_1
val_data = data"""


# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

print ("Im here2")

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=True)
        self.query = nn.Linear(n_embd, head_size, bias=True)
        self.value = nn.Linear(n_embd, head_size, bias=True)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

print ("Im here3")


class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

start_time = time.time()
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
end_time = time.time()

# Cheking time
training_time = end_time - start_time
print(f"Total training time: {training_time:.2f} seconds")

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
#open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

Im here2
Im here3
3.667225 M parameters
step 0: train loss 3.2124, val loss 3.2164
step 500: train loss 0.9881, val loss 0.9164
step 1000: train loss 0.9048, val loss 0.8208
step 1499: train loss 0.8495, val loss 0.7764
Total training time: 827.11 seconds

G4sG4sF4sd5sd5sF4sd5ed5eG5sF5sG5sF5eF5ed5sd5eRD4sF5sF5sF4sd5eG5sd4sF5sG3sC3sa3sG4sg4sF4sG4sF4sF4eRF4sd4sF4sd5eRD4sc4sa4sg4sRg4sg4sg4sa4ed4sd4eF4ed4sc4sF4ed4sD4eRG4sD4sG4sF5sdg5sd4sF4sd4sF4sd4eF4sRG4sd4sd4sF4sF4sF5sF5ed5eF5sF5sF5sF5sF5sF5eRF5sF5sF54sd4sF5sd4sd4sRF4sF4sF4sG4sF4sd4sd4sF5sF4sd4sa4sa4sG4sG4sRa4sG4sF4sa4sg4sa4sG4sg4sC4sa4sF4sd4sF4sd4sG4sF4sF5sF4sF4d4sF4sF4sF4sG4swRd4eA4sG4sF4sRF4qRF4sd4hRD4sd4sd4sd4ed4sd4sG4sF5sG4sF4sF4eRF4sf4sd4sF4sF4sF4sF4eg4sRa3hRF4sF4sd4ed3sd3sd3ed3sa3eRd3qRa3sF4s


In [ ]:
import random

def random_baseline_loss(test_data):
    total_loss = 0
    for i in range(len(test_data) - 1):
        random_prediction = random.choice(range(vocab_size))
        actual = test_data[i + 1].item()

        # Calculate the loss
        loss = -torch.log(torch.tensor(1 / vocab_size))
        total_loss += loss

        if i in [0, 500, 999]:  # Print at steps 0, 500, and 999
            print(f"Step {i}: Random Prediction = {random_prediction}, Actual = {actual}, Loss = {loss.item()}")

    average_loss = total_loss / len(test_data)
    return average_loss

random_baseline_loss(val_data)



Step 0: Random Prediction = 20, Actual = 3, Loss = 3.2188758850097656
Step 500: Random Prediction = 4, Actual = 21, Loss = 3.2188758850097656
Step 999: Random Prediction = 20, Actual = 8, Loss = 3.2188758850097656


tensor(3.2261)